In [16]:
%load_ext lab_black

from puzzles import load_lines, load

from enum import Enum, IntEnum
from collections import namedtuple


class Pointer:
    def __init__(self, n: int = 0):
        self.n = n


class Parameter(IntEnum):
    REFERENCE = 0
    VALUE = 1

    def read(self, cmd, n):
        return cmd[cmd[n]] if self.value == 0 else cmd[n]

    def write(self, cmd, n, x):
        if self.value == 0:
            cmd[cmd[n]] = x
        else:
            cmd[n] = x


Parameters = namedtuple("Parameters", ["first", "second", "third"])


def summate(codes, pointer, modes):
    a = modes.first.read(codes, pointer.n + 1)
    b = modes.second.read(codes, pointer.n + 2)
    modes.third.write(codes, pointer.n + 3, a + b)
    pointer.n += 4
    return True


def multiply(codes, pointer, modes):
    a = modes.first.read(codes, pointer.n + 1)
    b = modes.second.read(codes, pointer.n + 2)
    modes.third.write(codes, pointer.n + 3, a * b)
    pointer.n += 4
    return True


def inputs(codes, pointer, modes):
    # Parameters that an instruction writes to will never be in immediate mode.
    assert modes.first == Parameter.REFERENCE
    x = int(input("Insert a number:"))
    modes.first.write(codes, pointer.n + 1, x)
    pointer.n += 2
    return True


def outputs(codes, pointer, modes):
    print(modes.first.read(codes, pointer.n + 1))
    pointer.n += 2
    return True


def halt(codes, pointer, modes):
    # print("halt")
    return False


# def move(pointer, shift):
#     pointer.n += shift


def parse_opcode(codes, pointer):
    code = f"{codes[pointer.n]:05d}"
    op = CODES_POLICIES[int(code[-2:])]
    modes = Parameters(*[Parameter(int(x)) for x in code[:3][::-1]])
    return op, modes


CODES_POLICIES = {
    1: summate,
    2: multiply,
    3: inputs,
    4: outputs,
    99: halt,
}


def run_program(cds):
    p = Pointer(0)
    while True:
        op, modes = parse_opcode(cds, p)
        should_continue = op(cds, p, modes)
        if not should_continue:
            break
    return cds

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [17]:
(
    run_program([1, 9, 10, 3, 2, 3, 11, 0, 99, 30, 40, 50])
    == [3500, 9, 10, 70, 2, 3, 11, 0, 99, 30, 40, 50]
)

True

In [18]:
run_program([1, 0, 0, 0, 99]) == [2, 0, 0, 0, 99]

True

In [19]:
run_program([2, 3, 0, 3, 99]) == [2, 3, 0, 6, 99]

True

In [20]:
run_program([2, 4, 4, 5, 99, 0]) == [2, 4, 4, 5, 99, 9801]

True

In [8]:
run_program([1, 1, 1, 4, 99, 5, 6, 0, 99]) == [30, 1, 1, 4, 2, 5, 6, 0, 99]

True

In [11]:
run_program([1101, 100, -1, 4, 0])

[1101, 100, -1, 4, 99]

In [12]:
codes = [int(x) for x in load(5).strip().split(",")]
_ = run_program(codes)

Insert a number: 1


0
0
0
0
0
0
0
0
0
7566643


---

In [13]:
full_stop = False

for noun in range(0, 100):
    if full_stop:
        break

    for verb in range(0, 100):
        codes = [int(x) for x in load(2).strip().split(",")]
        codes[1] = noun
        codes[2] = verb
        if run_program(codes)[0] == 19690720:
            print(100 * noun + verb)
            full_stop = True
            break

5741


---